In [1]:
import pandas as pd

base = pd.read_csv('dataset.csv')

In [2]:
import numpy as np

previsores = base.iloc[:1400000,0:20].values
previsores = np.reshape(previsores, (2000, 700, 20))
print(previsores.shape)

classes = base.iloc[:1400000, 28:31].values
classes = np.reshape(classes, (2000, 700, 3))
print(classes.shape)

(2000, 700, 20)
(2000, 700, 3)


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, CuDNNLSTM, Bidirectional, Masking

In [6]:
def criarRede():
    model = Sequential()
  
    #model.add(Masking(mask_value = 0, input_shape = (700, 20)))
  
    model.add(Bidirectional(CuDNNLSTM(100, return_sequences = True), input_shape = (700, 20)))
    model.add(Dropout(0.3))
    
    model.add(Bidirectional(CuDNNLSTM(100, return_sequences = True)))
    model.add(Dropout(0.3))

    model.add(Bidirectional(CuDNNLSTM(100, return_sequences = True)))
    model.add(Dropout(0.3))
    
    model.add(Bidirectional(CuDNNLSTM(100, return_sequences = True)))
    model.add(Dropout(0.3))

    model.add(Bidirectional(CuDNNLSTM(100, return_sequences = True)))
    model.add(Dropout(0.3))
    
    model.add(Bidirectional(CuDNNLSTM(100, return_sequences = True)))
    model.add(Dropout(0.3))
    
    model.add(Bidirectional(CuDNNLSTM(100, return_sequences = True)))
    model.add(Dropout(0.3))
    
    model.add(Bidirectional(CuDNNLSTM(100, return_sequences = True)))
    model.add(Dropout(0.3))
  
    model.add(Dense(3, activation = 'softmax'))
  
    model.compile(optimizer = 'adam', metrics = ['acc'], loss='categorical_crossentropy')
  
    return model

In [7]:
accu = []
precisions = []
recalls = []
f1 = []

In [8]:
from sklearn.metrics import balanced_accuracy_score, classification_report, precision_score, recall_score, f1_score, confusion_matrix

def train_and_evaluate_model(model, x_train, y_train, x_test, y_test):
    model.fit(x_train, y_train, epochs = 100, verbose = 1, batch_size = 32)
  
    predicted = model.predict(x_test)
  
    y_teste = []
    predict = []
  
    predicted = np.reshape(predicted, (predicted.shape[0] * predicted.shape[1], 3))
    y_test = np.reshape(y_test, (y_test.shape[0] * y_test.shape[1], 3))
    x_test = np.reshape(x_test, (x_test.shape[0] * x_test.shape[1], 20))

    for i in range(len(x_test)):
        cont = 0
        for j in range(len(x_test[i])):
            cont += x_test[i][j]
        if cont != 0:
            y_teste.append(y_test[i])
            predict.append(predicted[i])
    
    y_teste = np.asarray(y_teste)
    predict = np.asarray(predict)

    predicted = predict
    y_test = y_teste

    predicted = np.argmax(predicted, axis=1)
    y_test = np.argmax(y_test, axis=1)

    accu.append(balanced_accuracy_score(y_test, predicted)) 
    precisions.append(precision_score(y_test, predicted, average = 'weighted'))
    recalls.append(recall_score(y_test, predicted, average = 'weighted'))
    f1.append(f1_score(y_test, predicted, average = 'weighted'))
    
    print(classification_report(y_test, predicted))
    print('Acurácia')
    print(balanced_accuracy_score(y_test, predicted))
    print('Precisao')
    print(precision_score(y_test, predicted, average = 'weighted'))
    print('Recall')
    print(recall_score(y_test, predicted, average = 'weighted'))
    print('F1')
    print(f1_score(y_test, predicted, average = 'weighted'))
    print(confusion_matrix(y_test, predicted))

In [9]:
from sklearn.model_selection import KFold

kf = KFold(n_splits = 5, shuffle = True)

for train_index, test_index in kf.split(previsores):
    model = None
    model = criarRede()
    print("TRAIN:", train_index, "TEST:", test_index)
    train_and_evaluate_model(model, previsores[train_index], classes[train_index],
                           previsores[test_index], classes[test_index])

W0820 16:01:41.383086 12172 deprecation.py:506] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0820 16:01:41.388074 12172 deprecation.py:506] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0820 16:01:41.390068 12172 deprecation.py:506] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\init_ops.py

TRAIN: [   0    2    3 ... 1996 1997 1998] TEST: [   1    5   11   13   19   20   26   36   38   42   53   58   59   60
   62   63   65   66   68   69   74   77   78   79   88   90   91   92
  106  115  118  125  131  138  139  143  154  164  165  171  176  182
  184  199  210  216  217  220  243  248  254  256  258  265  266  272
  276  278  297  299  301  309  321  324  335  340  347  354  355  356
  361  362  366  370  372  375  378  379  386  389  390  397  401  404
  408  409  413  417  418  419  423  428  430  432  435  438  463  468
  470  471  483  488  489  497  499  501  509  510  515  520  524  530
  542  544  553  556  558  561  562  570  572  573  574  576  586  599
  612  613  624  625  626  629  630  632  641  642  643  644  645  646
  650  652  658  662  684  690  696  698  701  707  710  716  717  718
  720  729  730  733  740  756  767  772  773  774  778  779  782  787
  792  799  805  806  809  810  826  833  838  844  848  854  855  868
  875  876  878  889  891  8

W0820 16:01:43.696584 12172 deprecation.py:323] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/100
1600/1600 [==============================] - 26s 16ms/sample - loss: 0.3739 - acc: 0.7041
Epoch 2/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.3429 - acc: 0.8266
Epoch 3/100
1600/1600 [==============================] - 22s 13ms/sample - loss: 0.3214 - acc: 0.8508
Epoch 4/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.3098 - acc: 0.8583
Epoch 5/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.3032 - acc: 0.8615
Epoch 6/100
1600/1600 [==============================] - 22s 13ms/sample - loss: 0.3036 - acc: 0.8578
Epoch 7/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.3001 - acc: 0.8633
Epoch 8/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.2955 - acc: 0.8660
Epoch 9/100
1600/1600 [==============================] - 22s 14ms/sample - loss: 0.2915 - acc: 0.8676
Epoch 10/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 

1600/1600 [==============================] - 21s 13ms/sample - loss: 0.1273 - acc: 0.9368
Epoch 81/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.1173 - acc: 0.9388
Epoch 82/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.1133 - acc: 0.9420
Epoch 83/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.1110 - acc: 0.9457
Epoch 84/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.1086 - acc: 0.9464
Epoch 85/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.1080 - acc: 0.9457
Epoch 86/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.1075 - acc: 0.9406
Epoch 87/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.1062 - acc: 0.9428
Epoch 88/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.1061 - acc: 0.9440
Epoch 89/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.10

...

Epoch 1/100
1600/1600 [==============================] - 23s 15ms/sample - loss: 0.3739 - acc: 0.7892
Epoch 2/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.3421 - acc: 0.8253
Epoch 3/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.3232 - acc: 0.8502
Epoch 4/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.3106 - acc: 0.8582
Epoch 5/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.3051 - acc: 0.8610
Epoch 6/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.3006 - acc: 0.8634
Epoch 7/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.3032 - acc: 0.8616
Epoch 8/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.2966 - acc: 0.8662
Epoch 9/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.2956 - acc: 0.8662
Epoch 10/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 

1600/1600 [==============================] - 21s 13ms/sample - loss: 0.1100 - acc: 0.9505
Epoch 81/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.1087 - acc: 0.9511
Epoch 82/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.1076 - acc: 0.9501
Epoch 83/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.1070 - acc: 0.9507
Epoch 84/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.1055 - acc: 0.9520
Epoch 85/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.1048 - acc: 0.9526
Epoch 86/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.1048 - acc: 0.9529
Epoch 87/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.1033 - acc: 0.9531
Epoch 88/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.1033 - acc: 0.9536
Epoch 89/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.10

1600/1600 [==============================] - 21s 13ms/sample - loss: 0.1992 - acc: 0.9140
Epoch 35/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.1950 - acc: 0.9164
Epoch 36/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.1921 - acc: 0.9172
Epoch 37/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.1870 - acc: 0.9194
Epoch 38/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.1883 - acc: 0.9189
Epoch 39/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.1808 - acc: 0.9211
Epoch 40/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.1761 - acc: 0.9232
Epoch 41/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.1730 - acc: 0.9251
Epoch 42/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.1705 - acc: 0.9263
Epoch 43/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.16

Epoch 1/100
1600/1600 [==============================] - 23s 15ms/sample - loss: 0.3732 - acc: 0.7345
Epoch 2/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.3500 - acc: 0.8312
Epoch 3/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.3282 - acc: 0.8475
Epoch 4/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.3145 - acc: 0.8562
Epoch 5/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.3084 - acc: 0.8598
Epoch 6/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.3045 - acc: 0.8617
Epoch 7/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.2998 - acc: 0.8627
Epoch 8/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.2981 - acc: 0.8635
Epoch 9/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.2978 - acc: 0.8650
Epoch 10/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 

1600/1600 [==============================] - 21s 13ms/sample - loss: 0.1138 - acc: 0.9473
Epoch 81/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.1127 - acc: 0.9480
Epoch 82/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.1108 - acc: 0.9494
Epoch 83/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.1106 - acc: 0.9501
Epoch 84/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.1100 - acc: 0.9499
Epoch 85/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.1084 - acc: 0.9500
Epoch 86/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.1070 - acc: 0.9506
Epoch 87/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.1067 - acc: 0.9503
Epoch 88/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.1066 - acc: 0.9511
Epoch 89/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.10

In [10]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_32 (Bidirectio (None, 700, 200)          97600     
_________________________________________________________________
dropout_32 (Dropout)         (None, 700, 200)          0         
_________________________________________________________________
bidirectional_33 (Bidirectio (None, 700, 200)          241600    
_________________________________________________________________
dropout_33 (Dropout)         (None, 700, 200)          0         
_________________________________________________________________
bidirectional_34 (Bidirectio (None, 700, 200)          241600    
_________________________________________________________________
dropout_34 (Dropout)         (None, 700, 200)          0         
_________________________________________________________________
bidirectional_35 (Bidirectio (None, 700, 200)         

In [11]:
print('Acurácias total')
print(accu)
accu = np.array(accu)
print(accu.mean())
print('Precision total')
print(precisions)
precisions = np.array(precisions)
print(precisions.mean())
print('Recalls total')
print(recalls)
recalls = np.array(recalls)
print(recalls.mean())
print('F1 total')
print(f1)
f1 = np.array(f1)
print(f1.mean())

Acurácias total
[0.6832005056168039, 0.6858149953376965, 0.6739785245941897, 0.682233909275694, 0.6925378459088584]
0.6835531561466485
Precision total
[0.6925185761934991, 0.7004274134651741, 0.6852703576932863, 0.6942133754812613, 0.7062615874519644]
0.695738262057037
Recalls total
[0.6932422171602126, 0.7005893724362299, 0.6855588506392017, 0.6952874690579609, 0.706638714185884]
0.6962633246958978
F1 total
[0.6927735079587924, 0.6998623134169389, 0.6852186775356133, 0.6940581643961096, 0.7059224200985141]
0.6955670166811936
